In [1]:
a=[0,1,2,3,4,5]
a[:3]

[0, 1, 2]

In [2]:
1

1

In [1]:
class Node:
    def __init__(self, isleaf=False):
        self.isleaf=isleaf
        self.keys=[]
        self.child=[]
    def __repr__(self):
        return f'k:{self.keys}'
    def split(self):
        length = len(self.keys)
        temp = Node(self.isleaf)
        mid = self.keys[length//2]
        temp.keys = self.keys[length//2+1:]
        self.keys = self.keys[:length//2]
        if self.child:
            temp.child = self.child[length//2+1:]
            self.child = self.child[:length//2+1]
        return self,mid,temp
        
# nn=Node()
# nn.keys=[10,20,30,40,50]
# nn.child=[[1,2],[12,13],[22,23],[33,39],[45,49],[55,60]]
# ll,mid,rr=nn.split()
# ll,ll.child , rr, rr.child

In [2]:
import random

class BTree:
    def __init__(self, m):
        self.m=m #차수 -> m개의 child를 가질 수 있음
        self.root=Node(True)
        # self.root.isleaf = 'root'
        # self.root=None

    # k : Key 
    # def insert_node(self,p_pos,p_node,node,key):
    def insert_node(self,node,key):
        """ 데이터 추가하는것, Root부터 시작해서 내려가도록 Trigger가 발동되며, 
        현재의 노드(node)가 leaf가 아닐경우, 재귀적으로 child를 내려감. 
        리프에서의 삽입이 이뤄지고나면, 그대로 자기자신만 반환하고, 
        Leaf에서의 삽입이 OverFlow가 일어날경우 중간key와 자기자신을 반환
        삽입을 하고난 이후, 반환되는 key가 있다면, 해당 child에서 OverFlow가 난것이므로, 관련연산 진행
        """
        pos=0
        while pos < len(node.keys):
            # print(pos)
            if key[0]==node.keys[pos][0]:
                #Duplicate
                print(f"Duplicates key {key}")
                return node, None 
            elif key[0]<node.keys[pos][0]: break
            pos+=1
        # print(key," Node : ",node , pos)
        
        # Position 찾기 완료
        if node.isleaf==True : # Leaf라면?? -> Leaf에서의 삽입 로직
            i=len(node.keys)-1
            node.keys.append([1e9,None])
            while i>=0 and key[0]<node.keys[i][0]:
                node.keys[i+1]=node.keys[i]
                i-=1
            node.keys[i+1]=key
            if len(node.keys) >= self.m : # 오버플로 발생?
                if node == self.root:
                    # print("Root OverFlow")
                    ll,mid,rr=node.split()
                    self.root=Node(False)
                    self.root.keys.append(mid)
                    self.root.child.append(ll)
                    self.root.child.append(rr)
                    return self.root, None
                else:
                    # print("Leaf OVERFLOW",node)
                    return node, node.keys[(self.m//2)]

            else: return node, None
        # elif node.isleaf == False: #Leaf가 아니라면?
        else: #Leaf가 아니라면?
            node.child[pos], child_mid_kv = self.insert_node(node.child[pos],key)
            if child_mid_kv: # Child에서 OverFlow발생 -> 자기에 추가시켜줘야해
                ll,mid,rr=node.child[pos].split()

                i=len(node.keys)-1
                node.keys.append([1e9,None])
                node.child.append([])
                while i>=0 and mid[0] < node.keys[i][0]:
                    node.keys[i+1]=node.keys[i]
                    node.child[i+2]=node.child[i+1]
                    i-=1
                node.keys[i+1]=mid
                # i=len(node.child)-1
                node.child[i+2]=rr
                if len(node.keys) >= self.m : # 오버플로 발생?
                    if node == self.root:
                        # print("Root OverFlow")
                        ll,mid,rr=node.split()
                        self.root=Node(False)
                        self.root.keys.append(mid)
                        self.root.child.append(ll)
                        self.root.child.append(rr)
                        return self.root, None
                if len(node.keys)>=self.m:
                    return node, node.keys[(self.m//2)]
            return node, None

    # def search_key_with_Parent(self,p_node,node,key):
    #     """ Find Key 
    #     IF FIND : RETURN [Node, Key]
    #     else : RETURN [None, None] """
    #     # print(node)
    #     pos=0
    #     while pos < len(node.keys):
    #         if key[0] == node.keys[pos][0]:
    #             # print(f'Find! key : {node.keys[pos]}')
    #             return p_node, node, key
    #         elif key[0] < node.keys[pos][0] : break
    #         else:
    #             pos+=1
    #     # print(node, pos)
    #     if node.child:
    #         return self.search_key(node, node.child[pos],key)
    #     else : 
    #         return None, None, None
    def search_key(self,node,key):
        """ Find Key 
        IF FIND : RETURN [Node, Key]
        else : RETURN [None, None] """
        # print(node)
        pos=0
        while pos < len(node.keys):
            if key[0] == node.keys[pos][0]:
                # print(f'Find! key : {node.keys[pos]}')
                return node, key
            elif key[0] < node.keys[pos][0] : break
            else:
                pos+=1
        # print(node, pos)
        if node.child:
            return self.search_key(node.child[pos],key)
        else : 
            return None, None
            # return f"NOT FOUND {key}"

    def print_inorder(self,node):
        if node.child:
            for i in range(len(node.keys)):
                self.print_inorder(node.child[i])
                print(node.keys[i])
            self.print_inorder(node.child[-1])
        else:
            for i in node.keys:
                print(i)
            # print(node.keys)
    
    def delete(self,node,key):
        pos = 0
        while pos<len(node.keys) and key[0] > node.keys[pos][0]:
            pos+=1
        if node.isleaf==True : # Leaf 에서 키를 찾든 못찾든
            if pos<len(node.keys) and key[0]==node.keys[pos][0]:
                node.keys.pop(pos)
                return "DELETE FROM LEAF COMPLETED" # 삭제연산완료표시
            else 
                return "NOT FOUND" # 리프까지 갔는데 못찾음
        else: # Leaf가 아닌 경우
            if pos < len(node.keys):
                if key[0]==node.keys[pos][0]:
                    # 리프가 아닌데 찾은경우
                    pass ## TODO : Internal Node Deletion Process
                else: # 리프가 아닌데 키값도 아닌데 pos는찾은경우 => 아래로 내려가자
                    self.delete(node.child[pos],key)
            
            # if len(node.child[pos].keys)
                



# if __name__=="__main__":
# print("Start")
# kvs = [[1,11],[7,77],[5,55],[6,66],[4,44],[3,33],[2,22],[23,9]]


      20  ,  40
10,15   25,30   50

In [3]:
bt = BTree(5)
for i in range(1,50):
    bt.insert_node(bt.root, [i,i*10])
# bt.insert_node(bt.root,[100,2])
bt.print_inorder(bt.root)

[1, 10]
[2, 20]
[3, 30]
[4, 40]
[5, 50]
[6, 60]
[7, 70]
[8, 80]
[9, 90]
[10, 100]
[11, 110]
[12, 120]
[13, 130]
[14, 140]
[15, 150]
[16, 160]
[17, 170]
[18, 180]
[19, 190]
[20, 200]
[21, 210]
[22, 220]
[23, 230]
[24, 240]
[25, 250]
[26, 260]
[27, 270]
[28, 280]
[29, 290]
[30, 300]
[31, 310]
[32, 320]
[33, 330]
[34, 340]
[35, 350]
[36, 360]
[37, 370]
[38, 380]
[39, 390]
[40, 400]
[41, 410]
[42, 420]
[43, 430]
[44, 440]
[45, 450]
[46, 460]
[47, 470]
[48, 480]
[49, 490]


In [14]:
bt.delete(bt.root,[37,0])

4

In [5]:
bt.root,'---', bt.root.child

(k:[[9, 90], [18, 180], [27, 270], [36, 360]],
 '---',
 [k:[[3, 30], [6, 60]],
  k:[[12, 120], [15, 150]],
  k:[[21, 210], [24, 240]],
  k:[[30, 300], [33, 330]],
  k:[[39, 390], [42, 420], [45, 450]]])

In [107]:
bt.root.child[0].child , bt.root.child[1].child

([k:[[3, 30], [6, 60]], k:[[12, 120], [15, 150]], k:[[21, 210], [24, 240]]],
 [k:[[30, 300], [33, 330]],
  k:[[39, 390], [42, 420]],
  k:[[48, 480], [51, 510]]])

In [123]:
bt.search_key(bt.root,[8,None])[0].isleaf


Find! key : [8, 80]


True